In [19]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import torch
from torch import optim
from torch.autograd import Variable as V
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

player_stat_00_01 = pd.read_csv("play_per_game_stat/00-01.csv",delimiter = ",")        
player_stat_01_02 = pd.read_csv("play_per_game_stat/01-02.csv",delimiter = ",")
player_stat_02_03 = pd.read_csv("play_per_game_stat/02-03.csv",delimiter = ",")
player_stat_03_04 = pd.read_csv("play_per_game_stat/03-04.csv",delimiter = ",")
player_stat_04_05 = pd.read_csv("play_per_game_stat/04-05.csv",delimiter = ",")
player_stat_05_06 = pd.read_csv("play_per_game_stat/05-06.csv",delimiter = ",")
player_stat_06_07 = pd.read_csv("play_per_game_stat/06-07.csv",delimiter = ",")
player_stat_07_08 = pd.read_csv("play_per_game_stat/07-08.csv",delimiter = ",")
player_stat_08_09 = pd.read_csv("play_per_game_stat/08-09.csv",delimiter = ",")
player_stat_09_10 = pd.read_csv("play_per_game_stat/09-10.csv",delimiter = ",")

player_stat_10_11 = pd.read_csv("play_per_game_stat/10-11.csv",delimiter = ",")
player_stat_11_12 = pd.read_csv("play_per_game_stat/11-12.csv",delimiter = ",")
player_stat_12_13 = pd.read_csv("play_per_game_stat/12-13.csv",delimiter = ",")
player_stat_13_14 = pd.read_csv("play_per_game_stat/13-14.csv",delimiter = ",")
player_stat_14_15 = pd.read_csv("play_per_game_stat/14-15.csv",delimiter = ",")
player_stat_15_16 = pd.read_csv("play_per_game_stat/15-16.csv",delimiter = ",")
player_stat_16_17 = pd.read_csv("play_per_game_stat/16-17.csv",delimiter = ",")

stat_00_10 = [player_stat_00_01,player_stat_01_02,player_stat_02_03,player_stat_03_04,player_stat_04_05,
             player_stat_05_06,player_stat_06_07,player_stat_07_08,player_stat_08_09,player_stat_09_10]
              
stat_10_17 = [ player_stat_10_11,player_stat_11_12,player_stat_12_13,
              player_stat_13_14,player_stat_14_15,player_stat_15_16,player_stat_16_17] 


mvp_train_dataframe = pd.read_csv("mvp/mvp_train.csv",delimiter = ",")
mvp_test_dataframe = pd.read_csv("mvp/mvp_test.csv",delimiter = ",")

def initialize_data(stat,mvp_dataframe):
    a=[]
    mvp_list = []
    dataframe = pd.read_csv("play_per_game_stat/template.csv",delimiter = ",")
    dataframe = dataframe.drop(["Pos","Tm","Rk"],axis=1)
    for i in range(len(stat)):
        a.append(stat[i].drop(["Pos","Tm","Rk"],axis=1))
        column_num = a[i].shape[0]
        a[i].fillna(0,inplace = True)
        for j in range(column_num):
            if(a[i].loc[j,"Player"].find("*") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("*")]
            if(a[i].loc[j,"Player"].find("\\") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("\\")]
            if(mvp_dataframe.loc[i,"name"] == a[i].loc[j,"Player"]):
                mvp_list.append(0)
            else:
                mvp_list.append(1)
        dataframe = dataframe.append(a[i])
        dataframe = dataframe.reset_index(drop=True)  # 来改变index就可以了,
    dataframe_with_player_name = dataframe
    dataframe = dataframe.drop(["Player"],axis=1)
    dataframe = pd.DataFrame(dataframe,dtype=np.float)
    return (dataframe_with_player_name,dataframe,mvp_list)

dataframe_00_10_with_player_name, dataframe_00_10 , mvp_list_train  = initialize_data(stat_00_10,mvp_train_dataframe)
print("|"*50)
dataframe_10_17_with_player_name ,dataframe_10_17, mvp_list_test = initialize_data(stat_10_17,mvp_test_dataframe)


print(len(mvp_list_test))
print(len(mvp_list_train))
print("10-17 mvps:")
for i in range(1632):
    if(mvp_list_test[i] == 0):
        print(dataframe_10_17_with_player_name.loc[i,"Player"])
#         print(dataframe_10_17_with_player_name.loc[i,"Player"] , i)
print("00-10 mvps:")
for i in range(1831):
    if(mvp_list_train[i] == 0):
        print(dataframe_00_10_with_player_name.loc[i,"Player"])        
    

||||||||||||||||||||||||||||||||||||||||||||||||||
1632
1831
10-17 mvps:
Derrick Rose
LeBron James
LeBron James
Kevin Durant
Stephen Curry
Stephen Curry
Russell Westbrook
00-10 mvps:
Allen Iverson
Tim Duncan
Tim Duncan
Kevin Garnett
Steve Nash
Steve Nash
Dirk Nowitzki
Kobe Bryant
LeBron James
LeBron James


In [28]:
X_train = torch.tensor(np.array(dataframe_00_10),dtype=torch.float) 
y_train = torch.tensor(mvp_list_train,dtype=torch.long)

X_test = torch.tensor(np.array(dataframe_10_17),dtype=torch.float) 
y_test = torch.tensor(mvp_list_test,dtype=torch.long)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.dtype)
print(y_train.dtype)
print(X_test.dtype)
print(y_test.dtype)

class Model(torch.nn.Module): 
    def __init__(self):
        super(Model, self).__init__() 
        self.linear1 = torch.nn.Linear(26, 10) 
        self.linear2 = torch.nn.Linear(10, 2) 
        self.sigmoid = torch.nn.Sigmoid()  #和上节的nn.function下的不一样
#         self.activate = torch.nn.Relu()
    def forward(self, x):
        x = self.sigmoid(self.linear1(x)) 
        x = self.sigmoid(self.linear2(x)) 
        return x

def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=250, lr=0.01, weight_decay=0):
    train_dataset = TensorDataset(X_train, y_train)
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
    # loss function
    loss_func = F.cross_entropy
    # optimizer
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    # training loop
    for epoch in range(epochs):
        for xb, yb in train_data_loader:
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        y_pred = model(X_test)
        acc = accuracy_score(torch.argmax(y_pred, dim=1).detach().numpy(), y_test)
        print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))
    
    for item in y_pred:
        print(item)
#     print(y_pred)


    
model = Model()
train_model(model, X_train, y_train, X_test, y_test, lr=0.01)



torch.Size([1831, 26])
torch.Size([1831])
torch.Size([1632, 26])
torch.Size([1632])
torch.float32
torch.int64
torch.float32
torch.int64
Loss at epoch 0 : 0.715752, test acc: 0.004289
Loss at epoch 1 : 0.706501, test acc: 0.580270
Loss at epoch 2 : 0.694923, test acc: 0.705882
Loss at epoch 3 : 0.680188, test acc: 0.963235
Loss at epoch 4 : 0.662448, test acc: 0.995711
Loss at epoch 5 : 0.650022, test acc: 0.995711
Loss at epoch 6 : 0.638403, test acc: 0.995711
Loss at epoch 7 : 0.627504, test acc: 0.995711
Loss at epoch 8 : 0.617675, test acc: 0.995711
Loss at epoch 9 : 0.608819, test acc: 0.995711
Loss at epoch 10 : 0.600670, test acc: 0.995711
Loss at epoch 11 : 0.593023, test acc: 0.995711
Loss at epoch 12 : 0.585755, test acc: 0.995711
Loss at epoch 13 : 0.578798, test acc: 0.995711
Loss at epoch 14 : 0.572113, test acc: 0.995711
Loss at epoch 15 : 0.565676, test acc: 0.995711
Loss at epoch 16 : 0.559471, test acc: 0.995711
Loss at epoch 17 : 0.553483, test acc: 0.995711
Loss at ep

tensor([0.1012, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1018, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8868], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8868], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8860], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8866], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8869], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8866], grad_fn=<SelectBackward>)
tensor([0.1015, 0.8868], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1015, 0.8866], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1015, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8869], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8869], grad_f

tensor([0.1012, 0.8872], grad_fn=<SelectBackward>)
tensor([0.1114, 0.8909], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1267, 0.8955], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1218, 0.8943], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1271, 0.8962], grad_fn=<SelectBackward>)
tensor([0.1267, 0.8958], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1231, 0.8946], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1268, 0.8955], grad_fn=<SelectBackward>)
tensor([0.1227, 0.8946], grad_fn=<SelectBackward>)
tensor([0.1267, 0.8958], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8871], grad_f

tensor([0.1013, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1041, 0.8881], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8870], grad_fn=<SelectBackward>)
tensor([0.1267, 0.8957], grad_fn=<SelectBackward>)
tensor([0.1264, 0.8954], grad_fn=<SelectBackward>)
tensor([0.1196, 0.8935], grad_fn=<SelectBackward>)
tensor([0.1016, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1217, 0.8943], grad_fn=<SelectBackward>)
tensor([0.1268, 0.8958], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1013, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1020, 0.8875], grad_fn=<SelectBackward>)
tensor([0.1232, 0.8947], grad_fn=<SelectBackward>)
tensor([0.1014, 0.8871], grad_fn=<SelectBackward>)
tensor([0.1228, 0.8946], grad_fn=<SelectBackward>)
tensor([0.1269, 0.8960], grad_fn=<SelectBackward>)
tensor([0.1267, 0.8957], grad_fn=<SelectBackward>)
tensor([0.1012, 0.8871], grad_f

In [16]:
bb = torch.tensor([0,2,1])
print(bb.shape)

torch.Size([3])
